Примеры сравнений через эмбеддинги

In [6]:
import torch
from transformers import AutoTokenizer, AutoModel

In [4]:
def pool(hidden_state, mask, pooling_method="mean"):
    if pooling_method == "mean":
        s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
        d = mask.sum(axis=1, keepdim=True).float()
        return s / d
    elif pooling_method == "cls":
        return hidden_state[:, 0]
    
def l2_normalize(x, dim=1, eps=1e-12):
    norm = torch.linalg.norm(x, ord=2, dim=dim, keepdim=True)
    return x / (norm + eps)

In [7]:
# Загрузка модели
tokenizer = AutoTokenizer.from_pretrained("sergeyzh/BERTA")
model = AutoModel.from_pretrained("sergeyzh/BERTA")

In [8]:
prefix = "categorize_entailment:"

# Пример строк для сравнения
text1 = f"{prefix} Бумага цветная Апплика двухсторонняя (А4, 16 листов, 8 цветов, офсетная)"
text2 = f"{prefix} Бумага цветная Апплика двустор. А4 16 л. 8цв. офсетная"

# Токенизация
tokenized = tokenizer([text1, text2], padding=True, truncation=True, return_tensors="pt")

In [9]:
# Получение эмбеддингов
with torch.no_grad():
    output = model(**tokenized)

emb = pool(output.last_hidden_state, tokenized["attention_mask"], pooling_method="mean")
emb = l2_normalize(emb, dim=1)

# Косинусное сходство
similarity = (emb[0] @ emb[1]).item()
print(f"Сходство между строками: {similarity:.4f}")

Сходство между строками: 0.9458


In [ ]:
from sentence_transformers import SentenceTransformer



In [22]:
list_a = [
    "Бумага цветная Апплика двухсторонняя (А4, 16 листов, 8 цветов, офсетная)"
]

list_b = [
    "Бумага цветная Апплика двустор. А4 16 л. 8цв. офсетная"
]

In [ ]:
sts_prefix = "categorize_entailment:"
encode_kwargs = {"normalize_embeddings": True}
if sts_prefix:
    encode_kwargs["prompt"] = sts_prefix

# Добавляем префикс к каждой строке
sentences_a = [sts_prefix + s for s in list_a]
sentences_b = [sts_prefix + s for s in list_b]

# Получаем эмбеддинги
embeddings_a = model.encode(sentences_a, **encode_kwargs)
embeddings_b = model.encode(sentences_b, **encode_kwargs)

# Вычисляем косинусные сходства по парам
similarities = (embeddings_a * embeddings_b).sum(axis=1)

# Вывод
for i, sim in enumerate(similarities):
    print(f"Сходство между парой {i}: {sim:.4f}")

Сходство между парой 0: 0.9585


In [4]:
import torch
from sentence_transformers import SentenceTransformer, util

# Проверяем, доступен ли CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется устройство: {device}")

# Загрузка предобученной модели для получения эмбеддингов
model = SentenceTransformer("sergeyzh/BERTA").to(device)

Используется устройство: cuda


Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.


In [ ]:
def find_similar_categories(product_name, categories, sts_prefix):
    """
    Возвращает список категорий, семантическая близость которых к названию товара.
    
    :param product_name: str - название товарной позиции
    :param categories: list[str] - список категорий
    :param sts_prefix
    :return: list[tuple(category, similarity)]
    """
    encode_kwargs = {"normalize_embeddings": True}
    if sts_prefix:
        encode_kwargs["prompt"] = sts_prefix

    # Получаем эмбеддинги
    product_embedding = model.encode(product_name, convert_to_tensor=True, **encode_kwargs)
    category_embeddings = model.encode(categories, convert_to_tensor=True, **encode_kwargs)

    # Вычисляем косинусное сходство
    similarities = util.cos_sim(product_embedding, category_embeddings)[0]

    # Фильтруем сортируем
    results = [(categories[i], float(similarities[i])) for i in range(len(categories))]
    results.sort(key=lambda x: x[1], reverse=True)

    return results

In [12]:
# Пример использования
product_name = "Набор кастрюль Лысьвенские эмали Urban flower эмалированная сталь 3 штуки (1.45 л, 2.9 л, 4.3 л)"
with open("..\\classify\\categories.txt", encoding="utf-8") as file:
    categories = file.read().split("\n")


In [6]:
import json
METRICS_CLASSIFY_DATASET_PATH = "..\\classify\\correct_categories\\approved_categories.json"

metrics_data = []

with open(METRICS_CLASSIFY_DATASET_PATH, "r", encoding="UTF-8") as file:
    for value in json.load(file).values():
        metrics_data += value["data"]


Используем "categorize:"

In [25]:

results_categorize = []
for product in metrics_data:
    results_categorize.append(find_similar_categories(product["title"], categories, sts_prefix="categorize:"))


In [30]:
metrics_data[0]["title"]

'Подставка для ноутбука/планшета Ugreen LP339 (40291_)'

In [29]:
results_categorize[0]

[('Аксессуары для мобильных устройств', 0.4211483597755432),
 ('Компьютеры и периферийное оборудование', 0.3450010120868683),
 ('Оптические приборы', 0.32344427704811096),
 ('Техника бытовая электронная', 0.3074308931827545),
 ('Мебель', 0.30487924814224243),
 ('Спорттовары', 0.279451459646225),
 ('Продукция для улицы', 0.27621158957481384),
 ('Инструменты и оборудование', 0.2743482291698456),
 ('Продукция для защиты', 0.26875388622283936),
 ('Продукция для обслуживания и ремонта', 0.2663242518901825),
 ('Продукция упаковочная', 0.26621192693710327),
 ('Канцелярские товары', 0.26115211844444275),
 ('Продукция для офиса', 0.2589397132396698),
 ('Продукция для людей с ограниченными возможностями', 0.25654393434524536),
 ('Изделия из пластмасс', 0.25627756118774414),
 ('Оборудование для оборонной промышленности', 0.25625622272491455),
 ('Оборудование электрическое прочее', 0.2480703592300415),
 ('Напитки', 0.24486041069030762),
 ('Инструменты', 0.24045993387699127),
 ('Обувь', 0.240330934

In [26]:
threshold = 0.2
counter = 0
count_categories = []
for product, matches in zip(metrics_data, results_categorize):
    
    match_categories = set([category.split(".")[0] for category, score in matches if score > threshold])
    count_categories.append(len(match_categories))
    if product["answer"] in match_categories:
        counter += 1
    # else:
    #     print(product["title"])
    #     print(*match_categories, sep="\n")
    #     print()

print(sum(count_categories) / len(count_categories))
print(max(count_categories))
print(min(count_categories))
print(counter)

21.030741410488247
68
0
447


Используем "categorize_entailment"

In [27]:

results_categorize_entailment = []
for product in metrics_data:
    results_categorize_entailment.append(find_similar_categories(product["title"], categories, sts_prefix="categorize_entailment:"))


In [28]:
threshold = 0.2
counter = 0
count_categories = []
for product, matches in zip(metrics_data, results_categorize_entailment):
    
    match_categories = set([category.split(".")[0] for category, score in matches if score > threshold])
    count_categories.append(len(match_categories))
    if product["answer"] in match_categories:
        counter += 1
    # else:
    #     print(product["title"])
    #     print(*match_categories, sep="\n")
    #     print()

print(sum(count_categories) / len(count_categories))
print(max(count_categories))
print(min(count_categories))
print(counter)

67.39783001808318
76
8
546
